# Example BEM and FEM forward calculation on Colin

In [ ]:
import os, numpy as np
from collections import OrderedDict   
from pyhemo.DUNEuroHead import DUNEuroHead
from pyhemo.OpenMEEGHead import OpenMEEGHead

In [ ]:
DATADIR = os.path.join('/', 'examples', 'colin')

In [ ]:
# IO helper function
def load_elecs_dips_txt(fn):
    with open(fn, 'r') as f:
    	elecs = [[float(i) for i in line.split()] for line in f.readlines()]
    return elecs

Common settings:

In [ ]:
# Setting conductivity values in S/m
cond = {'air': 2.5*pow(10, -14), 'gray': 0.33, 'white': 0.33, 'csf': 1.78, 'skull':  0.004125, 'scalp': 0.25} 
# Load electrode positions
sensors = load_elecs_dips_txt(os.path.join(DATADIR, 'electrodes_aligned.txt'))

Create BEM head instance

In [ ]:
# Constructor needs conductivity, meshes, electrode positions
from pyhemo.data_io import load_tri
conductivity = {'scalp': cond['scalp'], 'skull': cond['skull'], 'csf': cond['csf'], 'cortex': cond['white']}
geometry = OrderedDict([('cortex', load_tri(os.path.join(DATADIR, 'cortex.tri'))), 
                        ('csf', load_tri(os.path.join(DATADIR, 'csf.tri'))), 
                        ('skull', load_tri(os.path.join(DATADIR, 'skull.tri'))), 
                        ('scalp', load_tri(os.path.join(DATADIR, 'scalp.tri')))])
bem = OpenMEEGHead(conductivity, geometry, sensors)

Create FEM instance of mesh from MRIsegmentation (6 tissue types)

In [ ]:
mesh_filename = os.path.join(DATADIR, 'mesh6_maxvoxvol5.msh')
fem6 = DUNEuroHead(cond, mesh_filename, sensors)

Create FEM instance (of the 4 nested meshes above)

In [ ]:
mesh_filename = os.path.join(DATADIR, 'bnd4_1922_FEM.msh')
fem4 = DUNEuroHead(conductivity, mesh_filename, sensors)

Add dipoles to heads

In [ ]:
# Load sourcemodel from segmentation
dipoles = load_elecs_dips_txt(os.path.join(DATADIR, 'sourcemodel2000.txt'))
# Choose 10 dipoles (due to time constraints)
dipoles = np.array(dipoles)[np.random.choice(len(dipoles), 10)]
# Add dipoles to head
bem.add_dipoles(dipoles)
fem4.add_dipoles(dipoles)
fem6.add_dipoles(dipoles)

Calculate BEM leadfields

In [ ]:
# This takes approximately 2min
import time
a = time.time()
V_bem = {'dsm': bem.V('dsm')}
print('\nCalculated BEM leadfields in %f min.\n' % ((time.time()-a)/60))

Calculate FEM leadfields

In [ ]:
# This usually is very quick - approx 1-2min - due to the small mesh size (117k)
b = time.time()
V_fem4 = {}
for sm_type in ['Partial integration', 'Venant', 'Subtraction', 'Spatial Venant']:
    V_fem4[sm_type] = fem4.V(sm_type)
V_fem6 = {}
for sm_type in ['Partial integration', 'Venant', 'Subtraction', 'Spatial Venant']:
    V_fem6[sm_type] = fem6.V(sm_type)
print('\nCalculated FEM leadfields in %f min.\n' % ((time.time()-b)/60))

Compare FEM and BEM results

In [ ]:
print('Calculate Pearson correlation of BEM with different FEMs of (FEM6):')
for sm_type in ['Partial integration', 'Venant', 'Subtraction', 'Spatial Venant']:
    c = np.mean([np.abs(np.corrcoef(V_bem['dsm'][:,i], V_fem6[sm_type][:,i])[0,1]) for i in range(10)])
    print('%s: %f' % (sm_type, c))

In [ ]:
print('Calculate Pearson correlation of BEM with different FEMs of (FEM4):')
for sm_type in ['Partial integration', 'Venant', 'Subtraction', 'Spatial Venant']:
    c = np.mean([np.abs(np.corrcoef(V_bem['dsm'][:,i], V_fem4[sm_type][:,i])[0,1]) for i in range(10)])
    print('%s: %f' % (sm_type, c))